In [1]:
%matplotlib inline
from nuscenes.nuscenes import NuScenes
from nuscenes.nuscenes import RadarPointCloud
from nuscenes.utils.geometry_utils import view_points, box_in_image, BoxVisibility, transform_matrix

import operator
import time

import cv2
import numpy as np
import open3d as o3d
from mayavi import mlab
import matplotlib.pyplot as plt
from matplotlib.axes import Axes

from pyquaternion import Quaternion

from transformers import AutoImageProcessor, DetrForObjectDetection
import torch
from PIL import Image,ImageDraw
import requests


Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2023-04-17 23:21:19.121614: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
nusc = NuScenes(version='v1.0-mini', 
                dataroot='/home/charles/myDataSet/nuScenes/v1.0-mini', verbose=True)

Loading NuScenes tables for version v1.0-mini...
23 category,
8 attribute,
4 visibility,
911 instance,
12 sensor,
120 calibrated_sensor,
31206 ego_pose,
8 log,
10 scene,
404 sample,
31206 sample_data,
18538 sample_annotation,
4 map,
Done loading in 0.429 seconds.
Reverse indexing ...
Done reverse indexing in 0.1 seconds.


In [3]:
image_processor = AutoImageProcessor.from_pretrained("facebook/detr-resnet-50")
model = DetrForObjectDetection.from_pretrained("facebook/detr-resnet-50")


Could not find image processor class in the image processor config or the model config. Loading based on pattern matching with the model's feature extractor configuration.
The `max_size` parameter is deprecated and will be removed in v4.26. Please specify in `size['longest_edge'] instead`.
INFO - 2023-04-17 23:21:21,984 - helpers - Loading pretrained weights from url (https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-rsb-weights/resnet50_a1_0-14fe96d1.pth)


In [4]:
device = torch.device("cuda")
model.to(device)
model.device

device(type='cuda', index=0)

In [5]:
scene = nusc.scene[0] 

# 获取该场景的first token
cur_sample_info = nusc.get('sample', scene['first_sample_token'])
sensor_data_token = nusc.get('sample_data', cur_sample_info['data']['CAM_FRONT'])
img_path_ = nusc.get_sample_data_path(cur_sample_info['data']['CAM_FRONT'])
image = Image.open(img_path_)
inputs = image_processor(images=image, return_tensors="pt")
inputs.to(device)
outputs = model(**inputs)

target_sizes = torch.tensor([image.size[::-1]])
results = image_processor.post_process_object_detection(outputs, threshold=0.4, target_sizes=target_sizes)[0]

draw = ImageDraw.Draw(image)
img = cv2.imread(img_path_)


for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(box)
    draw.rectangle(box, outline="#FF0000", width=2)
    
    img = cv2.rectangle(img, (int(box[0]), int(box[1])),
                             (int(box[2]),int(box[3])), (255, 0,0))
cv2.imshow('xxx', img)
cv2.waitKey(5000)
cv2.destroyAllWindows()
# image.show()

[850.04, 440.5, 862.52, 458.65]
[877.21, 468.31, 900.64, 491.08]
[754.04, 438.81, 763.41, 453.25]
[1165.78, 449.75, 1177.63, 469.21]
[1236.32, 484.38, 1253.96, 504.21]
[1461.19, 489.83, 1516.3, 514.65]
[970.17, 466.75, 1003.5, 507.48]
[1363.75, 483.9, 1434.09, 512.61]
[713.85, 463.66, 785.83, 532.43]
[847.89, 440.33, 859.61, 457.61]
[1304.54, 485.53, 1349.82, 508.2]
[1306.23, 484.63, 1356.8, 508.64]
[790.66, 466.21, 809.86, 507.85]
[1078.3, 451.05, 1089.86, 467.28]
[713.72, 464.88, 786.74, 531.81]
[786.18, 465.63, 805.46, 506.95]
[764.34, 438.06, 774.29, 452.65]
[1085.94, 452.11, 1096.92, 468.02]
[903.46, 468.74, 945.63, 491.02]
[759.24, 438.28, 768.95, 453.02]
[126.92, 222.65, 597.44, 628.87]
[754.14, 438.75, 763.94, 453.8]
[799.98, 467.13, 818.02, 507.56]
[900.75, 400.56, 912.49, 420.42]
[1384.16, 486.85, 1442.28, 514.15]
[974.98, 459.76, 1034.88, 515.42]
[885.32, 468.83, 909.02, 491.33]
[975.1, 458.21, 1037.35, 516.64]
[1003.71, 475.83, 1078.59, 534.79]
[909.76, 464.77, 946.91, 480.

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to tar

In [6]:
img = cv2.imread(img_path_)

for score, label, box in zip(results["scores"], results["labels"], results["boxes"]):
    box = [round(i, 2) for i in box.tolist()]
    print(box)
    draw.rectangle(box, outline="#FF0000", width=2)
    
    img = cv2.rectangle(img, (int(box[0]), int(box[1])),
                             (int(box[2]),int(box[3])), (255, 0,0))
    
    # cv2.putText(img, label, (int(box[0]), int(box[1])), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 0, 255), 2)
    
    
cv2.imshow('xxx', img)
cv2.waitKey(5000)
cv2.destroyAllWindows()

[850.04, 440.5, 862.52, 458.65]
[877.21, 468.31, 900.64, 491.08]
[754.04, 438.81, 763.41, 453.25]
[1165.78, 449.75, 1177.63, 469.21]
[1236.32, 484.38, 1253.96, 504.21]
[1461.19, 489.83, 1516.3, 514.65]
[970.17, 466.75, 1003.5, 507.48]
[1363.75, 483.9, 1434.09, 512.61]
[713.85, 463.66, 785.83, 532.43]
[847.89, 440.33, 859.61, 457.61]
[1304.54, 485.53, 1349.82, 508.2]
[1306.23, 484.63, 1356.8, 508.64]
[790.66, 466.21, 809.86, 507.85]
[1078.3, 451.05, 1089.86, 467.28]
[713.72, 464.88, 786.74, 531.81]
[786.18, 465.63, 805.46, 506.95]
[764.34, 438.06, 774.29, 452.65]
[1085.94, 452.11, 1096.92, 468.02]
[903.46, 468.74, 945.63, 491.02]
[759.24, 438.28, 768.95, 453.02]
[126.92, 222.65, 597.44, 628.87]
[754.14, 438.75, 763.94, 453.8]
[799.98, 467.13, 818.02, 507.56]
[900.75, 400.56, 912.49, 420.42]
[1384.16, 486.85, 1442.28, 514.15]
[974.98, 459.76, 1034.88, 515.42]
[885.32, 468.83, 909.02, 491.33]
[975.1, 458.21, 1037.35, 516.64]
[1003.71, 475.83, 1078.59, 534.79]
[909.76, 464.77, 946.91, 480.

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to target thread (0x55f6c8605fe0)

QObject::moveToThread: Current thread (0x55f6c8605fe0) is not the object's thread (0x55f6acc65ae0).
Cannot move to tar

: 